In [3]:
# imports 
from empath import Empath
lexicon = Empath()

## Steps for feature analysis aka violence metric


1. Take advantage of Empath package to obtain categories representatives of violence
    - a. Produce clean data locally thanks to results 
    - b. Preprocess as did in the paper to train Empath to ensure alignment with features found 
    - c. Perform Empath.lexicon.analyse with word counts, sorted_list (gives out cosine similarity between the vector of the category and the vector of the text given (need to reread that in paper))

2. Identify the most represented categories in the plot per genre, per year
3. Identify the most represented categories per movies classified as violent per ChatGPT 
4. Balance the frequencies of the words per their weight //TFID-matrix 
5. Our own logistic regression model : stantardise/normalise the train, test data 
6. Transform our plots in bags of words 
7. Side quest: Perform clustering on bags of words (idea: does it appear by itself group of violent movies)
8. Visualisation of number of words per movies, per genres 
9. If skewed, try logarithm and see if it becomes gaussian -> fit with logarithm 
10. Our own model : have p-value
Idea : try to replicate chatgpt predictions and explain them 

In [4]:
# Manual selection of hypothesised relevant categories 
categories = [
    "monster", "weapon", "negative_emotion", "pain", "terrorism", "breaking", 
    "disappointment", "anger", "torment", "shame", "emotional", "sadness", 
    "rage", "injury", "fire", "disgust", "war", "dominant_personality", 
    "fight", "deception", "neglect", "dominant_heirarchical", "violence"
]

for category in categories: 
    # Manual inspection of the words associated with all pre-build categories 
    print("Category =" + category, lexicon.cats.get(category))

Category =monster ['superhuman', 'myth', 'alien', 'ferocious', 'fable', 'abomination', 'extraterrestrial', 'hunt', 'hunter', 'monster', 'mutate', 'monstrous', 'mutation', 'undead', 'legend', 'devil', 'mystical', 'evil', 'superpower', 'enemy', 'witch', 'vampire', 'underworld', 'ravenous', 'bloodthirsty', 'folklore', 'rabid', 'predator', 'supernatural', 'deadly', 'wolf', 'demon', 'terrorize', 'extinct', 'demonic', 'unstoppable', 'rogue', 'sighting', 'vicious', 'mythological', 'inhuman', 'danger', 'feral', 'creature', 'werewolf', 'villain', 'fearsome', 'lore', 'mutant', 'savage', 'beast', 'dragon', 'mythical', 'transformation', 'immortal', 'ruthless', 'parasite', 'feared', 'legendary', 'mutated', 'hybrid', 'foe', 'cursed', 'reincarnation']
Category =weapon ['firepower', 'armory', 'shot', '.45', 'butcher', 'enemy', 'throwing', 'assassin', 'extinguisher', 'barricade', 'unarmed', 'shooter', 'stake', 'gunfire', 'warhead', 'turret', 'mechanism', 'bomb', 'firing', 'magnum', 'wield', 'archery', 